In [ ]:
#run every time
#mount your google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#run once
#download your kaggle config file from account and upload it to drive with following command.
from google.colab import files
files.upload()

In [ ]:
#run once
#install kaggle and synchronize it with your account with the help of config file
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#run once
#download any kaggle database to any location in google drive
#here download "ieee-fraud-detection" database at "/content/gdrive/My\Drive/kaggle/fraud-detection"
!kaggle competitions download -c ieee-fraud-detection -p /content/gdrive/My\Drive/kaggle/fraud-detection

In [ ]:
#run everytime
#changing the main directory of colab file 
import os
os.chdir('/content/drive/MyDrive/kaggle/fraud-detection')

FileNotFoundError: ignored

In [ ]:
#run once
#unzipping the data into the newly created data folder
!mkdir data
!unzip -q ieee-fraud-detection.zip -d data/

In [ ]:
#run everytime
#download correct version of xgboost from here "https://drive.google.com/file/d/1vyHsOalOLz7bNl2fvSvUKqgnBjivuHe4/view?usp=sharing"
#following command will update the xgboost version to bring it in runnable state
!pip uninstall xgboost
!pip install xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl

In [ ]:
#run everytime
#importing all the dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os,gc,re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve,auc,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

import time
plt.style.use('seaborn-whitegrid')
import warnings
warnings.simplefilter("ignore")
colors = [x['color'] for x in plt.rcParams['axes.prop_cycle']]


from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

In [ ]:
#run everytime
#installing/checking nvidia drivers
#if gives error try changing the runtime type to GPU
!nvidia-smi

In [ ]:
#run once
#get a look at the sizes of downloaded data files
for f in os.listdir('./data'):
  print(f.ljust(30) + str(round(os.path.getsize('./data/' + f) / 1000000, 2)) + 'MB')

So right now we will be working with the IEEE-CIS Fraud Detection which contains the data provided by Vesta Coporation. Vesta Corporation who is one of the forerunners in guaranteed e-commerce payment solutions provided this dataset. The database in itself has 4 files, transaction data for training and testing and identity data for training and testing. By looking at the size of our database we can see that the training and the testing database is more or less equally divided and that’s why there are quite really low chances of overfitting of our model. Now to get an overview of our database let us look into the head or what we call the initial values of our training database

In [ ]:
#run once
df_train_transaction = pd.read_csv('data/train_transaction.csv')
df_train_identity = pd.read_csv('data/train_identity.csv')
print(f'Shape of transaction train data: {df_train_transaction.shape}')
print(f'Shape of identity train data: {df_train_identity.shape}')
df_train_transaction.head()

In [ ]:
#run once
df_train_identity.head()

In [ ]:
#run once
df_test_transaction = pd.read_csv('data/test_transaction.csv')
df_test_identity = pd.read_csv('data/test_identity.csv')
df_test_transaction.head()

In [ ]:
#run once
df_test_identity.head()

It is important to note that all transactions do not have corresponding identity information. Actually we don't have access to exact information about columns and what they represent in the database. Mostly it is because of security purposes as we are handling transaction data but we would try to predict what kind of information is actually provided by the columns or the features of the database. Let us get familiarity with various columns present:

**Transaction Database**
*   Transaction id: Id related to the transaction
*   TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
* TransactionAMT: transaction payment amount in USD
* ProductCD [Categorical]: product code(the product for each transaction)
* Card 1–6 [Categorical]: payment card related information like card type, country etc
* Addr1, addr2 [Categorical]: address information
* Dist1,dist2: some distance information
* P_emaildomain [Categorical]: email domain of purchaser.
* R_emaildomain [Categorical]: email domain of the recipient.
* C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
* D1-D15: time delta, such as days between the previous transactions, etc.
* M1-M9 [Categorical]: match, such as names on card and address, etc.
* Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
**Identity Database**
* Transaction id: Id related to the transaction
* DeviceType [Categorical]: Type of device used for the transaction
* DeviceInfo [Categorical]: More information about device used
* id 1–38 [Categorical+numeric]: network connection information,browser information etc (id 12–38 are categorical information)



In [ ]:
#run once
# we can observe that ids have different name in test dataset, let us correct that.
id_cols = [col for col in df_test_identity.columns if col[0]+col[1] == 'id']
rename_cols = {i:'id_'+str(i[-2]+i[-1]) for i in id_cols}
df_test_identity = df_test_identity.rename(columns=rename_cols)


print(f'Shape of transaction train data: {df_test_transaction.shape}')
print(f'Shape of identity train data: {df_test_identity.shape}')

df_train_transaction.head(3)

In [ ]:
#run once
df_train_identity.head(3)

In [ ]:
#run once
#now it is important to combine transaction and id dataset to analyse data better and quickly

df_train_transaction = pd.read_csv('data/train_transaction.csv')
df_train_identity = pd.read_csv('data/train_identity.csv')

df_train = df_train_transaction.merge(df_train_identity,on=['TransactionID'],how='left')

df_test_transaction = pd.read_csv('data/test_transaction.csv')
df_test_identity = pd.read_csv('data/test_identity.csv')

id_cols = [col for col in df_test_identity.columns if col[0]+col[1] == 'id']
rename_cols = {i:'id_'+str(i[-2]+i[-1]) for i in id_cols}
df_test_identity = df_test_identity.rename(columns=rename_cols)
df_test = df_test_transaction.merge(df_test_identity,on=['TransactionID'],how='left')

df_train.to_csv('data/train_combined.csv',index=False)
df_test.to_csv('data/test_combined.csv',index=False)

In [ ]:
#run everytime
#importing the combined database directly
df_train = pd.read_csv('data/train_combined.csv')
df_test = pd.read_csv('data/test_combined.csv')
print(df_train.shape)
print(df_test.shape)

In [ ]:
#run once
df_train.head()

In [ ]:
#run once
df_test.head()

In [ ]:
#run once
#all the database have some missing values and it is always preffered to deal with them at initial stage.
#function to find out the columns with highest percentage of missing values

def top_missing_cols(df,n=10,thresh=80):
  dff = (df.isnull().sum()/df.shape[0])*100
  dff = dff.reset_index()
  dff.columns = ['col','missing_percent']
  dff = dff.sort_values(by=['missing_percent'],ascending=False).reset_index(drop=True)
  print(f'There are {df.isnull().any().sum()} columns in this dataset with missing values.')
  print(f'There are {dff[dff["missing_percent"] > thresh].shape[0]} columns with missing percent values than {thresh}%')
  if n:
    return dff.head(n)
  else:
    return dff

In [ ]:
#run once
top_missing_cols(df_train,thresh=90)

In [ ]:
#run once
top_missing_cols(df_test,thresh=90)

We can see that columns like id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_01,id_07,id_08 have more than 99 percent missing values. Its better we drop those features. Now we will start with the EDA of the dataset. It's not ideal to explain the analysis of each and every feature here. I will explain some of my interesting observations.

In [ ]:
#run once
#Analysis of Is_Fraud column. This is the column which we need to predict

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10,6))
sns.countplot(df_train['isFraud'])
axes.title.set_text('Target')
total = float(df_train['isFraud'].shape[0])  
for p in axes.patches:
    height = p.get_height()
    axes.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height*100/total),
            ha="center") 
plt.show()

**Observations**

As expected we can see that class is heavily imbalanced. Here 96.5% of transactions are not fraud and the rest 3.5% of transactions are fraudulent. We will choose Area under the ROC curve (AUC) as a metric for our ML problem. AUC is the area under the ROC curve. It tells how much the model is capable of distinguishing between classes. Higher the AUC, the better the model is at predicting 0s as 0s and 1s as 1s. The ROC curve is plotted with TPR against the FPR where TPR is on the y-axis and FPR is on the x-axis.


In [ ]:
#run once
#Analysis of Transaction Date. We will check if it is continuous or not. This is important for train test split.

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(12,5))
sns.histplot(df_train['TransactionDT'], color='g')
sns.histplot(df_test['TransactionDT'],color='r')
axes.title.set_text('Train - Test Transaction date - distribution')
plt.show()

**Observations:**
There is a slight gap in between, but otherwise, the training set is from an earlier period of time and test data from a later period of time. This can impact train validation split or cross-validation techniques should be used.

In [ ]:
#run once
#getting the percentage of fraud transaction per day of weak

df_train['dayofweek'] = (df_train['TransactionDT']//(60*60*24)-1)%7
tmp = df_train[['isFraud','dayofweek']].groupby(by=['dayofweek']).mean().reset_index() \
            .rename(columns={'isFraud':'Percentage fraud transactions'})

tmp_count = df_train[['TransactionID','dayofweek']].groupby(by=['dayofweek']).count().reset_index() \
            .rename(columns={'TransactionID':'Number of transactions'})
tmp = tmp.merge(tmp_count,on=['dayofweek'])




fig, axes = plt.subplots(figsize=(12,5))
axes = sns.lineplot(x=tmp['dayofweek'],y=tmp['Percentage fraud transactions'],color='r')
axes2 = axes.twinx()
axes2 = sns.barplot(x=tmp['dayofweek'],y=tmp['Number of transactions'],palette="flare")
axes.set_title('Fraud transaction vs dayofweek')
plt.show()

In [ ]:
#run once
#getting the percentage of fraud transaction per hour of week

df_train['hour'] = (df_train['TransactionDT']//(60*60))%24
tmp = df_train[['isFraud','hour']].groupby(by=['hour']).mean().reset_index() \
            .rename(columns={'isFraud':'Percentage fraud transactions'})

tmp_count = df_train[['TransactionID','hour']].groupby(by=['hour']).count().reset_index() \
            .rename(columns={'TransactionID':'Number of transactions'})
tmp = tmp.merge(tmp_count,on=['hour'])




fig, axes = plt.subplots(figsize=(12,5))
axes = sns.lineplot(x=tmp['hour'],y=tmp['Percentage fraud transactions'],color='r')
axes2 = axes.twinx()
axes2 = sns.barplot(x=tmp['hour'],y=tmp['Number of transactions'],palette='flare')
axes.set_title('Fraud transaction(no of transactions) vs hour')
plt.show()

**Observations:** We can see that on the 3rd fay fraudulent transactions are very less, similarly, in the 7th hour, the percentage of fraudulent transactions is high compared to other hours.

In [ ]:
#run once
#checking exceptionally high amount transactions

fig, axes = plt.subplots(1,2,figsize=(15,5))

sns.scatterplot(y=df_train['TransactionAmt'],x=df_train['TransactionDT'],hue=df_train['isFraud'],ax=axes[0])
axes[0].title.set_text('Transcation Amount - Train')

sns.scatterplot(y=df_test['TransactionAmt'],x=df_test['TransactionDT'],ax=axes[1])
axes[1].title.set_text('Transcation Amount - Test')

plt.show()

In [ ]:
#run once
#as most transaction are having low amount of money to get better insight let's try comparing the log of transactions

test_amt = np.log(df_test[['TransactionAmt']])

dff_fraud = df_train[df_train['isFraud'] == 1]
dff_notfraud = df_train[df_train['isFraud'] == 0]

dff_fraud['TransactionAmt'] = np.log(dff_fraud['TransactionAmt'])
dff_notfraud['TransactionAmt'] = np.log(dff_notfraud['TransactionAmt'])


fig,axes = plt.subplots(1,2,figsize=(15,8))
sns.distplot(dff_notfraud['TransactionAmt'],ax=axes[0],label='not fraud')
sns.distplot(dff_fraud['TransactionAmt'],ax=axes[0],label='fraud')
axes[0].title.set_text('Log(Fraud transaction distribution) Train')
axes[0].legend()

sns.distplot(test_amt,ax=axes[1])
axes[1].title.set_text('Log(Fraud transaction distribution) Test')
plt.show()

**Observations:** We can see that there is one point in train data where the amount > 30000. It's better we remove the outlier since it can affect our models(especially distance-based algorithms like logistic regression, knn etc.) in prediction. Also, Outliers like this can cause an overfitting problem. For instance, tree-based models can put these outliers in leaf nodes, which are noise and not part of a general pattern. Therefore, I decided to remove the values larger than 30,000 in the training set.

It seems that the transactions with ‘LogTransactionAmt’ larger than 5.5 (244 dollars) and smaller than 3.3 (27 dollars) have a higher frequency and probability density of being fraudulent. On the other hand, the ones with ‘LogTransactionAmt’ from 3.3 to 5.5 have a higher chance of being legit.

In [ ]:
#run everytime
#removing exceptionally high values from database

df_train = df_train[df_train['TransactionAmt'] < 30000]
df_train.shape

In [ ]:
#run once
#analysing the product type column

# idea from https://www.kaggle.com/gpreda/home-credit-default-risk-extensive-eda
def hor_plot(feat,df=df_train,label_rotation=False,shape=(12,8)):
    
    val_cnts = df[feat].value_counts()
    df1 = pd.DataFrame({feat: val_cnts.index,'Number of units': val_cnts.values})
    # Calculate the percentage of target=1 per category value.For that what we did is we took the mean value of TARGET
    percent = df[[feat, 'isFraud']].groupby([feat],as_index=False).mean()
    percent.sort_values(by='isFraud', ascending=False, inplace=True)
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=shape)
    s = sns.barplot(ax=ax1, x = feat, y="Number of units",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    s = sns.barplot(ax=ax2, x = feat, y='isFraud', order=percent[feat], data=percent)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    plt.ylabel('Percent of Fraud transactions [Target with value 1]', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=10)
    plt.show();


hor_plot(df=df_train,feat='ProductCD')

We can see that among transactions related to product code C about 12 percent are fraud transactions. similarly close to 6% of transactions are fraud among product codes.

We have D columns from D1 to D15. These represent time deltas such as days between previous transaction and current transaction, days between current and first transaction, etc. Even though they have not given a clear understanding regarding D cols, we got some insights after a good analysis. We assume D1 as the day since credit card usage has begun. Subtracting this from Transaction day will result in content value per client.

In [ ]:
#run once
#Observing D1 before normalization

plt.figure(figsize=(15,5))
plt.scatter(df_train['TransactionDT'],df_train['D1n'])
plt.title(f'Original {col}')
plt.xlabel('Time')
plt.ylabel(f'{col}')
plt.show()

In [ ]:
#run everytime
#normalizing D1

df_train['D1n'] =  df_train['D1'] - df_train.TransactionDT/np.float32(24*60*60)

In [ ]:
#run once
#Observing D1 after normalization

plt.figure(figsize=(15,5))
plt.scatter(df_train['TransactionDT'],df_train['D1n'])
plt.title(f'Original {col}')
plt.xlabel('Time')
plt.ylabel(f'{col}')
plt.show()

In [ ]:
#run once
#Observing Card1 to Card6

for c in cards:
    if df_train[c].dtypes in ['int64','float64']:
        fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10,8))
        sns.histplot(df_train[c])
        axes.title.set_text(f'Card {c}')
        plt.show()

In [ ]:
#run once
hor_plot('card4',label_rotation=True)

In [ ]:
#run once
hor_plot('card6')

**Observing Card1 - Card6**

These columns represent payment information related to cards. Some interesting observations from these features are:

* There is a wide range of values in card1. If we see the distribution there are overlapping's. card1 alone cannot distinguish between fraudulent and nonfraudulent transactions. No missing values are there in card1.
* A small percent of data is missing in card2. Similar to card1 a lot of unique values are present in card2 also. card3 has a low correlation with other card columns.
* card 4 indicates which type of card do client uses visa, Mastercard, American Express, or discover. Similarly, card6 indicates the type of card- debit or credit.

**Some Important Observations:**
* **P-email domain and R-email domain:** These are email domains of purchaser and recipient respectively. About 76% of values are missing in the R-email domain. In purchaser email domains most mails are from gmail.com. Among those More fraud transactions comes from domain protonmail.com which is more than 90% which is a serious issue.
* **addr1 and addr2:** These are some address-related information related to the client. Around 11% of data is missing from both addr1 and addr2. There are about 332 unique values in addr1 and 74 in the case of addr2.
* **dist1 and dist2:** This might indicate the distance between transaction location and card owner address. This is just my assumption. We have about 93% of values missing in dist2. There is no correlation between dist1 and dist2. While dist2 is very weakly correlated with all other attributes.
* **C1-C14:** These are information such as counting. C3 is somewhat different from other Cxx features. For c3, from box pot, it is clear that there is no value beyond 3 in train data for fraudulent transactions. For no fraudulent transactions, the value ranges from 0 to 26. Also, C3 is very weakly co-related with other Cxx columns.
* **M1-M9:** More than 50 percent of values are missing among M columns. These features do not clearly distinguish between fraudulent and nonfraudulent transactions.
* **V1-V399:** These are vesta-engineered features. These are masked information that can be ranking, counting, or other entity relations. There exist a strong correlation (>0.9) between many features. If possible It's better we reduce the number of these features as it can reduce the computational complexity of the model.

In [ ]:
#run everytime
#Differing category columns from numeric ones

# column details
cat_cols = (['ProductCD'] + 
            ['card%d' % i for i in range(1, 7)] + 
            ['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain'] + 
            ['M%d' % i for i in range(1, 10)] + 
            ['DeviceType', 'DeviceInfo'] +
            ['id_%d' % i for i in range(12, 39)])


type_map = {c: str for c in cat_cols}
df_train[cat_cols] = df_train[cat_cols].astype(type_map, copy=False)
df_test[cat_cols] = df_test[cat_cols].astype(type_map, copy=False)

######################################################################################

id_cols = ['TransactionID', 'TransactionDT']
target = 'isFraud'

numeric_cols =  [
    'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 
    'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 
    'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
    'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 
    'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 
    'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 
    'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 
    'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 
    'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 
    'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 
    'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 
    'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 
    'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 
    'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 
    'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 
    'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 
    'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 
    'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 
    'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 
    'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 
    'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 
    'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252', 
    'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 
    'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 
    'V277', 'V278', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 
    'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 
    'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 
    'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 
    'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 
    'V337', 'V338', 'V339', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 
    'id_09', 'id_10', 'id_11'
]

In [ ]:
#run everytime runningRandomForest 
## Train and test split¶
y_train_ = df_train['isFraud']
X_train = df_train.drop(columns=['isFraud'])
X_test = df_test.copy()

print(X_train.shape)
print(X_test.shape)
gc.collect()

In [ ]:
#it is usually preferred to reload the database for new model so we can delete recent one for now.

del df_train,df_test

In [ ]:
#run everytime running Random Forest
# Label encoding all cat features
for col in X_train.columns:
    if col in cat_cols:
        # label encode all cat columns
        dff = pd.concat([X_train[col],X_test[col]])
        dff,_ = pd.factorize(dff,sort=True)
        if dff.max()>32000: 
            print(col,'needs int32 datatype')
            
        X_train[col] = dff[:len(X_train)].astype('int16')
        X_test[col] = dff[len(X_train):].astype('int16')

In [ ]:
#run everytime running Random Forest
# Scaling numeric features
cols = X_train.columns
for col in cols:
    if col not in cat_cols and col not in id_cols:
        # min max scalar
        dff = pd.concat([X_train[col],X_test[col]])
        dff = (dff - dff.min())/(dff.max() - dff.min())
        dff.fillna(-1,inplace=True)

        X_train[col] = dff[:len(X_train)]
        X_test[col] = dff[len(X_train):]

del dff

In [ ]:
#run everytime running Random Forest
# # train test split
# from sklearn.model_selection import train_test_split
# x_train,x_cv_,y_train,y_cv = train_test_split(X_train,y_train, stratify = y_train,test_size=0.3, random_state=40)

idx_train = X_train.index[:int(X_train.shape[0]*0.75)]  
idx_validation = X_train.index[int(X_train.shape[0]*0.75):]

x_train,y_train = X_train.iloc[idx_train],y_train_.iloc[idx_train]
x_cv_,y_cv = X_train.iloc[idx_validation],y_train_.iloc[idx_validation]

x_train.to_csv('data/x_train.csv',index=False)
x_cv_.to_csv('data/x_cv_.csv',index=False)
y_train.to_csv('data/y_train.csv',index=False)
y_cv.to_csv('data/y_cv.csv',index=False)

print(x_train.shape)
print(x_cv_.shape)
print(y_train.shape)
print(y_cv.shape)

In [ ]:
#run everytime running the Random forest
#Actually running or fitting the model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_cv_)

train_fpr, train_tpr, thresholds = roc_curve(y_train, model.predict_proba(x_train)[:,1])
test_fpr, test_tpr, thresholds = roc_curve(y_cv, model.predict_proba(x_cv_)[:,1])

#Area under ROC curve
print('Area under train roc {}'.format(auc(train_fpr, train_tpr)))
print('Area under test roc {}'.format(auc(test_fpr, test_tpr)))

In [ ]:
#run everytime running Random Forest
#Due to lower accuracy trying to figure out the perfect depth for random trees in model
from sklearn.model_selection import RandomizedSearchCV
clf = RandomForestClassifier(n_jobs= -1,class_weight='balanced')
no_of_estimators = [5, 10, 50, 100, 120]
maximumdepth = [1, 5, 7, 10, 15, 25, 30]
parameters = {'n_estimators': no_of_estimators ,'max_depth':maximumdepth}
model = RandomizedSearchCV(estimator=clf,  param_distributions=parameters, cv=3, n_iter=6, scoring='roc_auc')
model.fit(x_train,y_train)

In [ ]:
#run everytime running Random Forest
model.best_params_

In [ ]:
#run everytime running Random Forest
#running the Random forest model again but with better parameters
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=15,n_estimators=10)
model.fit(x_train,y_train)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_cv_)

train_fpr, train_tpr, thresholds = roc_curve(y_train, model.predict_proba(x_train)[:,1])
test_fpr, test_tpr, thresholds = roc_curve(y_cv, model.predict_proba(x_cv_)[:,1])

#Area under ROC curve
print('Area under train roc {}'.format(auc(train_fpr, train_tpr)))
print('Area under test roc {}'.format(auc(test_fpr, test_tpr)))

In [ ]:
#run everytime running the random forest
#After getting nice accuracy, creating confusion matrix to analyse results better

train_cf = confusion_matrix(y_train,y_train_pred)
plt.figure(figsize=(7,5))
sns.heatmap(train_cf,annot=True,annot_kws={"size": 16},fmt="0")
plt.title('Train confusion matrix')
plt.show()


cv_cf = confusion_matrix(y_cv,y_test_pred)
plt.figure(figsize=(7,5))
sns.heatmap(cv_cf,annot=True,annot_kws={"size": 16},fmt="0")
plt.title('Test confusion matrix')
plt.show()

We already got our results with a decent accuracy with Random Forest model but to increase the accuracy let us do a coorelation analysis of database to understand how various features are dependent on each other, this will not only help us in increasing accuracy but will also help us in dropping less significant features. 

In [ ]:
#run everytime
#importing databases once again
df_train = pd.read_csv('data/train_combined.csv')
df_test = pd.read_csv('data/test_combined.csv')
print(df_train.shape)
print(df_test.shape)

In [ ]:
#run everytime
#A function to find out columns with missing values more than a particular %

def top_missing_cols(df,n=10,thresh=80):
    """
    returns missing columns in dataframe with missing values percent > thresh
    if n=None. It will gave whole dataframe with missing values percent > thresh
    """
    
    dff = (df.isnull().sum()/df.shape[0])*100
    dff = dff.reset_index()
    dff.columns = ['col','missing_percent']
    dff = dff.sort_values(by=['missing_percent'],ascending=False).reset_index(drop=True)
    print(f'There are {df.isnull().any().sum()} columns in this dataset with missing values.')
    print(f'There are {dff[dff["missing_percent"] > thresh].shape[0]} columns with missing percent values than {thresh}%')
    if n:
        return dff.head(n)
    else:
        return dff

In [ ]:
#run everytime
#we will only run Coorelation Analysis on columns having more than 50% missing values

df_missing = top_missing_cols(df_train,n=None,thresh=50)
# Taking all column with missing percen > 50
missing_cols = df_missing['col']

In [ ]:
#run everytime
# we will take all columns and group them based on missing percentage

nan_dict = {}
for col in missing_cols:
    count = df_train[col].isnull().sum()
    try:
        nan_dict[count].append(col)
    except:
        nan_dict[count] = [col]
        
for k,v in nan_dict.items():
    print(f'#####' * 4)
    print(f'NAN count = {k} percent: {(int(k)/df_train.shape[0])*100} %')
    print(v)

**Reducing V-Cols**

From our Xgboost model, after plotting feature importance we came to know that all V cols do not contribute much to the model. Also on analysis, we found that there exists a strong correlation between several ‘Vxx’ columns. So we decided to reduce the number of columns by doing a correlation analysis.

**Procedure**
* Group columns based on a number of missing values Eg. if there are 4 columns v1,v2,v3, and v4. If v1 and v3 have 56 missing values and v2 have 21 and v4 have 5 missing values, we have 3 groups [‘v1’,’v3'], [‘v2’] and [‘v4’]
* In each group, For each column in that group find the correlation with other columns and take only columns with a correlation coefficient > 0.75. Take the largest list with common elements as a subgroup. Each group contains several subgroups. For eg: if we have [[v1,v2],[v6],[v1,v4,v2,v5],[v5,v4]] ,our output will be [[v1,v2,v4,v5],[v6]]. Now from each subgroup choose the column with the most number of unique values. For eg, In subgroup [v1,v2,v4,v5], let v2 have the most unique values. So out output becomes [v2,v6].

We know that columns ‘V35’, ‘V40’, ‘V41’, ‘V39’, ‘V38’, ‘V51’, ‘V37’, ‘V52’, ‘V36’, ‘V50’, ‘V48’, ‘V42’,‘V43’, ‘V44’, ‘V46’, ‘V47’, ‘V45’, ‘V49’ 168969 missing values. So they form a group. Now we will plot a correlation matrix.

In [ ]:
#run everytime
#A function to get the coorelation matrix between a set of columns

def coorelation_analysis(cols,title='Coorelation Analysis',size=(12,12)):
    cols = sorted(cols)
    fig,axes = plt.subplots(1,1,figsize=size)
    df_corr = df_train[cols].corr()
    sns.heatmap(df_corr,annot=True,cmap='RdBu_r')
    axes.title.set_text(title)
    plt.show()

In [ ]:
#run everytime
#A function to get all the columns having significance lower than 0.75

def reduce_groups(grps):
    '''
    determining column that have more unique values among a group of atttributes
    '''
    use = []
    for col in grps:
        max_unique = 0
        max_index = 0
        for i,c in enumerate(col):
            n = df_train[c].nunique()
            if n > max_unique:
                max_unique = n
                max_index = i
        use.append(col[max_index])
    return use

In [ ]:
#run once
#An example of Coorelation Analysis

cols = ['V35', 'V40', 'V41', 'V39', 'V38', 'V51', 'V37', 'V52', 'V36', 'V50', 'V48', 'V42',
 'V43', 'V44', 'V46', 'V47', 'V45', 'V49']
coorelation_analysis(cols,title='Coorelation Analysis: V35-V52',size=(12,12))

In [ ]:
#run once
#Getting pairs which can be reduced from previous analysis

pairs = [['V35','V36'],['V37','V38'],['V39','V40','V42','V43','V50','V51','V52'],['V41'],
         ['V44','V45'],['V46','V47'],['V48','V49']]
red_cols = reduce_groups(pairs)
red_cols

In [ ]:
#run once
#After running Coorelation analysis over all VCols the columns that can be reduced are

reduced_vcols = ['V1', 'V3', 'V4', 'V6', 'V8', 'V11', 'V13', 'V14', 'V17', 'V20', 
 'V23', 'V26', 'V27', 'V30', 'V36', 'V37', 'V40', 'V41', 'V44', 'V47', 'V48', 'V54', 'V56', 'V59', 
 'V62', 'V65', 'V67', 'V68', 'V70', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89', 'V91', 'V96', 
 'V98', 'V99', 'V104', 'V107', 'V108', 'V111', 'V115', 'V117', 'V120', 'V121', 'V123', 'V124', 'V127', 
 'V129', 'V130', 'V136', 'V138', 'V139', 'V142', 'V147', 'V156', 'V162', 'V165', 'V160', 'V166', 'V178',
 'V176', 'V173', 'V182', 'V187', 'V203', 'V205', 'V207', 'V215', 'V169', 'V171', 'V175', 'V180', 'V185', 
 'V188', 'V198', 'V210', 'V209', 'V218', 'V223', 'V224', 'V226', 'V228', 'V229', 'V235', 'V240', 'V258', 
 'V257', 'V253', 'V252', 'V260', 'V261', 'V264', 'V266', 'V267', 'V274', 'V277', 'V220', 'V221', 'V234', 
 'V238', 'V250', 'V271', 'V294', 'V284', 'V285', 'V286', 'V291',
 'V297', 'V303', 'V305', 'V307', 'V309', 'V310', 'V320', 'V281', 'V283', 'V289', 'V296', 'V301', 'V314', 'V332', 'V325', 'V335', 'V338']

We did a similar coorelation analysis for ID, C Cols and M cols and created a list of all the columns needed to be reduced

In [ ]:
#run everytime running XGboost
# column details
cat_cols = (['ProductCD'] + 
            ['card%d' % i for i in range(1, 7)] + 
            ['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain'] + 
            ['M%d' % i for i in range(1, 10)] + 
            ['DeviceType', 'DeviceInfo'] +
            ['id_%d' % i for i in range(12, 39)])


type_map = {c: str for c in cat_cols}
df_train[cat_cols] = df_train[cat_cols].astype(type_map, copy=False)
df_test[cat_cols] = df_test[cat_cols].astype(type_map, copy=False)

######################################################################################

id_cols = ['TransactionID', 'TransactionDT']
target = 'isFraud'

numeric_cols =  [
    'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 
    'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 
    'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
    'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 
    'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 
    'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 
    'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 
    'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 
    'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 
    'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 
    'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 
    'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 
    'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 
    'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 
    'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 
    'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 
    'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 
    'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 
    'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 
    'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 
    'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 
    'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252', 
    'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 
    'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 
    'V277', 'V278', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 
    'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 
    'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 
    'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 
    'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 
    'V337', 'V338', 'V339', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 
    'id_09', 'id_10', 'id_11'
]


reduced_vcols = ['V1', 'V3', 'V4', 'V6', 'V8', 'V11', 'V13', 'V14', 'V17', 'V20', 
 'V23', 'V26', 'V27', 'V30', 'V36', 'V37', 'V40', 'V41', 'V44', 'V47', 'V48', 'V54', 'V56', 'V59', 
 'V62', 'V65', 'V67', 'V68', 'V70', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89', 'V91', 'V96', 
 'V98', 'V99', 'V104', 'V107', 'V108', 'V111', 'V115', 'V117', 'V120', 'V121', 'V123', 'V124', 'V127', 
 'V129', 'V130', 'V136', 'V138', 'V139', 'V142', 'V147', 'V156', 'V162', 'V165', 'V160', 'V166', 'V178',
 'V176', 'V173', 'V182', 'V187', 'V203', 'V205', 'V207', 'V215', 'V169', 'V171', 'V175', 'V180', 'V185', 
 'V188', 'V198', 'V210', 'V209', 'V218', 'V223', 'V224', 'V226', 'V228', 'V229', 'V235', 'V240', 'V258', 
 'V257', 'V253', 'V252', 'V260', 'V261', 'V264', 'V266', 'V267', 'V274', 'V277', 'V220', 'V221', 'V234', 
 'V238', 'V250', 'V271', 'V294', 'V284', 'V285', 'V286', 'V291',
 'V297', 'V303', 'V305', 'V307', 'V309', 'V310', 'V320', 'V281', 'V283', 'V289', 'V296', 'V301', 'V314', 'V332', 'V325', 'V335', 'V338']

In [ ]:
#run everytime running XGboost
#dropping columns to be reduced

drop_cols = [col for col in df_train.columns if col[0] == 'V' and col not in reduced_vcols]

print(f'dropping {len(drop_cols)} columns')
df_train = df_train.drop(columns=drop_cols)
df_test = df_test.drop(columns=drop_cols)

In [ ]:
#run evverytime running XGBoost
## Train and test split

y_train = df_train['isFraud']
X_train = df_train.drop(columns=['isFraud'])
X_test = df_test.copy()

print(X_train.shape)
print(X_test.shape)
gc.collect()

In [ ]:
#run everytime running XGBoost
# Label encoding all cat features

for col in X_train.columns:
    
    if col in cat_cols:
        # label encode all cat columns
        dff = pd.concat([X_train[col],X_test[col]])
        dff,_ = pd.factorize(dff,sort=True)
        if dff.max()>32000: 
            print(col,'needs int32 datatype')
            
        X_train[col] = dff[:len(X_train)].astype('int16')
        X_test[col] = dff[len(X_train):].astype('int16')

In [ ]:
#run everytime running XGBoost

rem_cols = []
rem_cols.extend(['TransactionDT','TransactionID'])

cols = [col for col in X_train.columns if col not in rem_cols]
len(cols)

In [ ]:
#run everytime running XGBoost
# Scaling numeric features

for col in cols:
    if col not in cat_cols:
        # min max scalar
        dff = pd.concat([X_train[col],X_test[col]])
        dff = (dff - dff.min())/(dff.max() - dff.min())
        dff.fillna(-1,inplace=True)

        X_train[col] = dff[:len(X_train)]
        X_test[col] = dff[len(X_train):]

del dff

In [ ]:
#run everytime running XGBoost
#Fitting the XGBoost model

x_train = X_train[cols]
x_test = X_test[cols]


idx_train = x_train.index[:int(x_train.shape[0]*0.75)]  
idx_validation = x_train.index[int(x_train.shape[0]*0.75):]
    
print(f'fitting model on {len(cols)} columns')
clf = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        tree_method='gpu_hist' 
          )
model = clf.fit(x_train.loc[idx_train,cols], y_train[idx_train], 
            eval_set=[(x_train.loc[idx_validation,cols],y_train[idx_validation])],
            verbose=50, early_stopping_rounds=100)

In [ ]:
#run everytime running XGBoost
#Making predictions with XGboost on training and testing database

y_train_pred = model.predict(x_train.iloc[idx_train])
y_test_pred = model.predict(x_train.iloc[idx_validation])

train_fpr, train_tpr, thresholds = roc_curve(y_train.iloc[idx_train], model.predict_proba(x_train.iloc[idx_train])[:,1])
test_fpr, test_tpr, thresholds = roc_curve(y_train.iloc[idx_validation], model.predict_proba(x_train.iloc[idx_validation])[:,1])

#Area under ROC curve
print('Area under train roc {}'.format(auc(train_fpr, train_tpr)))
print('Area under test roc {}'.format(auc(test_fpr, test_tpr)))

In [ ]:
#run everytime running XGBoost
#Creating Confusion matrix for analysis

train_cf = confusion_matrix(y_train.iloc[idx_train],y_train_pred)
plt.figure(figsize=(7,5))
sns.heatmap(train_cf,annot=True,annot_kws={"size": 16},fmt="0")
plt.title('Train confusion matrix')
plt.show()


cv_cf = confusion_matrix(y_train.iloc[idx_validation],y_test_pred)
plt.figure(figsize=(7,5))
sns.heatmap(cv_cf,annot=True,annot_kws={"size": 16},fmt="0")
plt.title('Test confusion matrix')
plt.show()

In [ ]:
#it is usually preferred to reload the database for new model so we can delete recent one for now.

del df_train,df_test

**Engineering D Cols**

From Kaggle's discussions, I came to know that D1 is the day since credit card usage began. Subtracting this from Transaction Day will make it an almost constant value per client. I did the same approach for all the D cols. To check the newly created features are helpful or not I did a forward feature selection.

In [ ]:
#run everytime
#importing database again

df_train_transaction = pd.read_csv('data/train_transaction.csv')
df_train_identity = pd.read_csv('data/train_identity.csv')
print(f'Shape of transaction train data: {df_train_transaction.shape}')
print(f'Shape of identity train data: {df_train_identity.shape}')

In [ ]:
#run everytime 

# column details
cat_cols = (['ProductCD'] + 
            ['card%d' % i for i in range(1, 7)] + 
            ['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain'] + 
            ['M%d' % i for i in range(1, 10)] + 
            ['DeviceType', 'DeviceInfo'] +
            ['id_%d' % i for i in range(12, 39)])


type_map = {c: str for c in cat_cols}
df_train[cat_cols] = df_train[cat_cols].astype(type_map, copy=False)
df_test[cat_cols] = df_test[cat_cols].astype(type_map, copy=False)

######################################################################################

id_cols = ['TransactionID', 'TransactionDT']
target = 'isFraud'

numeric_cols =  [
    'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 
    'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 
    'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
    'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 
    'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 
    'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 
    'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 
    'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 
    'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 
    'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 
    'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 
    'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 
    'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 
    'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 
    'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 
    'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 
    'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 
    'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 
    'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 
    'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 
    'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 
    'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252', 
    'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 
    'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 
    'V277', 'V278', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 
    'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 
    'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 
    'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 
    'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 
    'V337', 'V338', 'V339', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 
    'id_09', 'id_10', 'id_11'
]

v_cols_included = ['V1', 'V3', 'V4', 'V6', 'V8', 'V11', 'V13', 'V14', 'V17', 'V20', 
 'V23', 'V26', 'V27', 'V30', 'V36', 'V37', 'V40', 'V41', 'V44', 'V47', 'V48', 'V54', 'V56', 'V59', 
 'V62', 'V65', 'V67', 'V68', 'V70', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89', 'V91', 'V96', 
 'V98', 'V99', 'V104', 'V107', 'V108', 'V111', 'V115', 'V117', 'V120', 'V121', 'V123', 'V124', 'V127', 
 'V129', 'V130', 'V136', 'V138', 'V139', 'V142', 'V147', 'V156', 'V162', 'V165', 'V160', 'V166', 'V178',
 'V176', 'V173', 'V182', 'V187', 'V203', 'V205', 'V207', 'V215', 'V169', 'V171', 'V175', 'V180', 'V185', 
 'V188', 'V198', 'V210', 'V209', 'V218', 'V223', 'V224', 'V226', 'V228', 'V229', 'V235', 'V240', 'V258', 
 'V257', 'V253', 'V252', 'V260', 'V261', 'V264', 'V266', 'V267', 'V274', 'V277', 'V220', 'V221', 'V234', 
 'V238', 'V250', 'V271', 'V294', 'V284', 'V285', 'V286', 'V291',
 'V297', 'V303', 'V305', 'V307', 'V309', 'V310', 'V320', 'V281', 'V283', 'V289', 'V296', 'V301', 'V314', 'V332', 'V325', 'V335', 'V338']

In [ ]:
#run everytime
# droping v cols 

drop_cols = [col for col in df_train.columns if col[0] == 'V' and col not in v_cols_included]

print(f'dropping {len(drop_cols)} columns')
df_train = df_train.drop(columns=drop_cols)
df_test = df_test.drop(columns=drop_cols)

In [ ]:
#run everytime
## Train and test split

y_train = df_train['isFraud']
X_train = df_train.drop(columns=['isFraud'])
X_test = df_test.copy()

print(X_train.shape)
print(X_test.shape)
gc.collect()

In [ ]:
#run everytime
# Label encoding all cat features

for col in X_train.columns:
    if col in cat_cols:
        # label encode all cat columns
        dff = pd.concat([X_train[col],X_test[col]])
        dff,_ = pd.factorize(dff,sort=True)
        if dff.max()>32000: 
            print(col,'needs int32 datatype')
            
        X_train[col] = dff[:len(X_train)].astype('int16')
        X_test[col] = dff[len(X_train):].astype('int16')

TransactionDT and TransactionID,
Both columns are unique. One is time-related information and the other is a unique id. Adding this to the model doesn't make much sense. So we removed those features.

In [ ]:
#run everytime
#removing the 2 columns 

rem_cols = []
rem_cols.extend(['TransactionDT','TransactionID'])

In [ ]:
#run everytime
#defining XGBoost model with all the parameters

def model(x_train,y_train,cols):
    idx_train = x_train.index[:int(x_train.shape[0]*0.75)]  
    idx_validation = x_train.index[int(x_train.shape[0]*0.75):]
    
    print(f'fitting model on {len(current_cols)} columns')
    clf = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        tree_method='gpu_hist' 
          )
    model = clf.fit(x_train.loc[idx_train,cols], y_train[idx_train], 
            eval_set=[(x_train.loc[idx_validation,cols],y_train[idx_validation])],
            verbose=50, early_stopping_rounds=100)
    del clf
    return model.best_score

In [ ]:
#run everytime
# NORMALIZE D COLUMNS
for i in range(1,16):
    X_train['D'+str(i)+'n'] =  X_train['D'+str(i)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)+'n'] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60)

In [ ]:
#run everytime
# Scaling numeric features
cols = [col for col in X_train.columns if col not in rem_cols]
for col in cols:
    if col not in cat_cols:
        # min max scalar
        dff = pd.concat([X_train[col],X_test[col]])
        dff = (dff - dff.min())/(dff.max() - dff.min())
        dff.fillna(-1,inplace=True)

        X_train[col] = dff[:len(X_train)]
        X_test[col] = dff[len(X_train):]

del dff

In [ ]:
del df_train,df_test

In [ ]:
#run everytime
#replacing the old D cols with new normalized ones

d_norm_cols = ['D'+str(i)+'n' for i in range(1,16)]
d_cols = ['D'+str(i) for i in range(1,16)]
old_cols = [col for col in X_train.columns if col not in rem_cols and col not in d_norm_cols]
len(old_cols)

In [ ]:
#run everytime
#choosing the only required D cols and dropping the rest of them by checking accuracy of model with all

len_new_cols = 16
best_cols = []
best_score = 0.9231
iteration = 1


while iteration < len_new_cols:
    for i in range(15):
        print(f'Added new column {d_norm_cols[i]}')
        if d_norm_cols[i] not in best_cols:
            current_cols = old_cols.copy()
            # adding and removing Di
        
            best_cols.append(d_norm_cols[i])
            
            print(f'working with {best_cols}')

            current_cols.extend([c for c in best_cols])

            for c in best_cols:
              if c[:-1] in current_cols:  #if D1 in current cols
                current_cols.remove(c[:-1])

            
            print('--'*40)
            current_score = model(X_train,y_train,current_cols)

            if current_score > best_score:
                print(f'score improved on adding {d_norm_cols[i]} new best score {current_score}')
                iteration_best_col = d_norm_cols[i]
                best_score = current_score

            else:
                current_cols.remove(d_norm_cols[i])
                current_cols.append(d_norm_cols[i][:-1])

            best_cols.remove(d_norm_cols[i])
        else:
          print(f'Column {d_norm_cols[i]} already present')    

    
    iteration += 1
    if iteration_best_col is not None:
        best_cols.append(iteration_best_col)
        iteration_best_col = None
        print(f'Best col in this iteration {iteration_best_col}, current best cols {best_cols}, score {best_score}')
                  
    else:
        print('None of the columns improving the score in this iteration')
        break
    print('###'*40)


In [ ]:
 gc.collect()